Import & Combine

In [1]:
import pandas as pd
import concurrent.futures
import os

# Define the range of years
start_year = 2012
end_year = 2024

# Function to read a single Excel file
def read_excel_file(year):
    file_path = f"data/raw/contratospub{year}.xlsx"
    return pd.read_excel(file_path)

# Use ThreadPoolExecutor to read files in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Map the read_excel_file function to the range of years
    df_list = list(executor.map(read_excel_file, range(start_year, end_year + 1)))

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Display the combined DataFrame
combined_df.head()


# Save as pickle file
pkl_output_path = "data/processed/combined_data.pkl"
combined_df.to_pickle(pkl_output_path)

Columns wrangling

In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the combined DataFrame from the pickle file
pkl_input_path = "data/processed/combined_data.pkl"

# Check if the pickle file exists before loading
if os.path.exists(pkl_input_path):
    combined_df = pd.read_pickle(pkl_input_path)
else:
    print(f"File not found: {pkl_input_path}")

# Drop columns
combined_df = combined_df.drop(columns=['Observacoes', 'numAcordoQuadro', 'DescrAcordoQuadro', 'concorrentes', 'regime', 'fundamentacao', 'nAnuncio', 'TipoAnuncio', 'idINCM', 'ProcedimentoCentralizado', 'dataFechoContrato', 'linkPecasProc', 'CritMateriais', 'tipoFimContrato', 'justifNReducEscrContrato', 'dataPublicacao'])

# Separar a coluna adjudicante usando a primeira ocorrência de " - "
combined_df['nif_adjudicante'] = combined_df['adjudicante'].str.split(' - ', n=1, expand=True)[0]
combined_df['nome_adjudicante'] = combined_df['adjudicante'].str.split(' - ', n=1, expand=True)[1]

# Remover a coluna original
combined_df.drop(columns=['adjudicante'], inplace=True)

# Separar a coluna cpv em código e descrição
combined_df[['cpv_código', 'cpv_descricao']] = combined_df['cpv'].str.split(' - ', n=1, expand=True)

# Criar a coluna cpv_division a partir dos dois primeiros números do cpv_código
combined_df['cpv_division'] = combined_df['cpv_código'].str[:2]

# Remover a coluna original
combined_df.drop(columns=['cpv'], inplace=True)

# Função para processar a geografia
def process_geografia(entry):
    if not isinstance(entry, str):
        return 'Desconhecido', 'Desconhecido'
    # Separar múltiplos locais pelo separador "|"
    locais = entry.split(' | ')
    # Extrair apenas NUTS I e NUTS II
    nuts_ii = set()
    for local in locais:
        niveis = local.split(', ')
        if len(niveis) >= 2:
            nuts_ii.add(niveis[1])  # Pegamos no NUTS II
        else:
            nuts_ii.add('Portugal')  # Apenas Portugal (NUTS I)
    
    # Decidir o âmbito_geo
    if len(nuts_ii) == 1:
        if 'Portugal' in nuts_ii:
            return 'Portugal', 'Nacional'
        else:
            return list(nuts_ii)[0], list(nuts_ii)[0]
    else:
        return list(nuts_ii)[0], 'Múltiplos'

# Adicionar barra de progresso com tqdm
tqdm.pandas(desc="Processando geografia")

# Aplicar a função à coluna geografia com tqdm
combined_df[['NUTS', 'ambito_geo']] = combined_df['localExecucao'].progress_apply(process_geografia).apply(pd.Series)

# Resultado final
print(combined_df.head())

# Save as pickle file
pkl_output_path = "data/processed/portal_base_trimmed.pkl"
combined_df.to_pickle(pkl_output_path)

combine datasets portal & sioe

In [ ]:
import pandas as pd

sioe_df = pd.read_excel('/workspaces/portal_base/data/raw/ExportResultadosPesquisa20241219T113545438Z.xlsx')

# Drop columns: 'Contacto 4 -  Tipo', 'Contacto 4 -  Contacto' and 68 other columns
sioe_df = sioe_df.drop(columns=['Morada 1 -  Latitude', ' NISS',' Código de Certidão Permanente de Registo','Morada 1 -  Longitude', 'Morada 1 -  Altitude', 'Contacto 4 -  Tipo', 'Contacto 4 -  Contacto', 'Contacto 4 -  Utilização', 'Contacto 4 -  Principal', 'Contacto 5 -  Tipo', 'Contacto 5 -  Contacto', 'Contacto 5 -  Utilização', 'Contacto 5 -  Principal', 'Contacto 6 -  Tipo', 'Contacto 6 -  Contacto', 'Contacto 6 -  Utilização', 'Contacto 6 -  Principal', 'Contacto 7 -  Tipo', 'Contacto 7 -  Contacto', 'Contacto 7 -  Utilização', 'Contacto 7 -  Principal', 'Contacto - Link 2 -  URL', 'Contacto - Link 2 -  Descrição', 'Contacto - Link 2 -  Classificação', 'Contacto - Link 3 -  URL', 'Contacto - Link 3 -  Descrição', 'Contacto - Link 3 -  Classificação', 'Contacto - Link 4 -  URL', 'Contacto - Link 4 -  Descrição', 'Contacto - Link 4 -  Classificação', 'Contacto - Link 5 -  URL', 'Contacto - Link 5 -  Descrição', 'Contacto - Link 5 -  Classificação', 'Contacto - Link 6 -  URL', 'Contacto - Link 6 -  Descrição', 'Contacto - Link 6 -  Classificação', 'Contacto - Link 7 -  URL', 'Contacto - Link 7 -  Descrição', 'Contacto - Link 7 -  Classificação', 'Contacto - Link 8 -  URL', 'Contacto - Link 8 -  Descrição', 'Contacto - Link 8 -  Classificação', 'Contacto - Link 9 -  URL', 'Contacto - Link 9 -  Descrição', 'Contacto - Link 9 -  Classificação', 'Contacto - Link 10 -  URL', 'Contacto - Link 10 -  Descrição', 'Contacto - Link 10 -  Classificação', 'Contacto - Link 11 -  URL', 'Contacto - Link 11 -  Descrição', 'Contacto - Link 11 -  Classificação', 'Contacto - Link 12 -  URL', 'Contacto - Link 12 -  Descrição', 'Contacto - Link 12 -  Classificação', 'Contacto - Link 13 -  URL', 'Contacto - Link 13 -  Descrição', 'Contacto - Link 13 -  Classificação', 'Contacto - Link 14 -  URL', 'Contacto - Link 14 -  Descrição', 'Contacto - Link 14 -  Classificação', 'Contacto - Link 15 -  URL', 'Contacto - Link 15 -  Descrição', 'Contacto - Link 15 -  Classificação', 'Contacto - Link 16 -  URL', 'Contacto - Link 16 -  Descrição', 'Contacto - Link 16 -  Classificação', 'Contacto - Link 17 -  URL', 'Contacto - Link 17 -  Descrição', 'Contacto - Link 17 -  Classificação', 'Contacto - Link 18 -  URL', 'Contacto - Link 18 -  Descrição', 'Contacto - Link 18 -  Classificação','Contacto - Link 19 -  URL', 'Contacto - Link 19 -  Descrição', 'Contacto - Link 19 -  Classificação'])


# Separar a coluna adjudicante usando a primeira ocorrência de " - "
sioe_df['cae_codigo'] = sioe_df[' CAE'].str.split(' - ', n=1, expand=True)[0]
sioe_df['cae_atividade'] = sioe_df[' CAE'].str.split(' - ', n=1, expand=True)[1]
sioe_df.drop(columns=[' CAE'], inplace=True)

# Remover o sufixo '.0'
sioe_df[' NIPC'] = sioe_df[' NIPC'].astype(str)
sioe_df[' NIPC'] = sioe_df[' NIPC'].str.replace('.0', '', regex=False)

#salvar como pickle 
sioe_df.to_pickle("/workspaces/portal_base/data/processed/sioe_base.pkl")


In [1]:
import pandas as pd

portal_base_df = pd.read_pickle("/workspaces/portal_base/data/processed/portal_base_trimmed.pkl")

# import "/workspaces/portal_base/data/processed/sioe_base.pkl"
sioe_df = pd.read_pickle("/workspaces/portal_base/data/processed/sioe_base.pkl")

In [ ]:
# Realizar a junção (merge)
merged_df = pd.merge(
    portal_base_df,
    sioe_df,
    left_on=['nif_adjudicante', 'nome_adjudicante'],
    right_on=[' NIPC', ' Designação'],
    how='left',  # 'left' garante que mantemos todas as linhas de portal_base_df
    indicator=True
)

# Filtrar as linhas de portal_base_df sem correspondência
no_match_portal = merged_df[merged_df['_merge'] == 'left_only']

# Contar as linhas sem correspondência
num_no_match_portal = no_match_portal.shape[0]

print(f"Número de linhas de portal_base_df sem correspondência: {num_no_match_portal}")

# Verificar correspondência para 'nif_adjudicante' apenas
nif_merged_df = pd.merge(
    portal_base_df,
    sioe_df,
    left_on='nif_adjudicante',
    right_on=' NIPC',
    how='left',
    indicator=True
)

nif_no_match = nif_merged_df[nif_merged_df['_merge'] == 'left_only']
num_no_match_nif = nif_no_match.shape[0]

print(f"Número de linhas de portal_base_df sem correspondência em 'nif_adjudicante': {num_no_match_nif}")

# Verificar correspondência para 'nome_adjudicante' apenas
nome_merged_df = pd.merge(
    portal_base_df,
    sioe_df,
    left_on='nome_adjudicante',
    right_on=' Designação',
    how='left',
    indicator=True
)

nome_no_match = nome_merged_df[nome_merged_df['_merge'] == 'left_only']
num_no_match_nome = nome_no_match.shape[0]

print(f"Número de linhas de portal_base_df sem correspondência em 'nome_adjudicante': {num_no_match_nome}")




In [ ]:
# Realizar a junção para verificar correspondência apenas no 'nif_adjudicante'
nif_merged_df = pd.merge(
    portal_base_df,
    sioe_df,
    left_on='nif_adjudicante',
    right_on=' NIPC',
    how='left',
    indicator=True
)

# Filtrar as linhas de portal_base_df sem correspondência no 'nif_adjudicante'
nif_no_match = nif_merged_df[nif_merged_df['_merge'] == 'left_only']

# Criar o subset apenas com as colunas de portal_base_df originais
subset_no_match_nif = nif_no_match[portal_base_df.columns]